# library

In [1]:
import numpy as np, os
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# df_all

In [2]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# 2) MODEL
from keras.models import load_model
lstm2 = load_model('/project/LSH/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

Using TensorFlow backend.


## ReLU에는 inverse 해서 학습해서 구한 diff
## Sign에는 모두 1로 바꿔서 학습해서 구한 diff

In [4]:
# 3)
# FEATURES
a = pd.read_csv('/project/LSH/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 


preds2_mean_lst = []
results = []
preds = lstm2.predict(X_test)
preds_mean = np.mean(preds)

# COMPUTE BASELINE
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()
    X_test[:,:,k] = 1
#     X_test[:,:,k] = np.where(X_test[:,:,k]==1, 2, X_test[:,:,k])
#     X_test[:,:,k] = np.where(X_test[:,:,k]==0, 1, X_test[:,:,k])
#     X_test[:,:,k] = np.where(X_test[:,:,k]==2, 0, X_test[:,:,k])

    # COMPUTE BCE WITH FEATURE K REVERSED, preds2_mean_list
    preds2 = lstm2.predict(X_test)
#     preds2_mean = np.mean(preds2)
    preds2_mean_lst.append(np.mean(preds2))
    
    loss_bce = bce(y_test, preds2).numpy()
    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [33:47<00:00,  2.01it/s]


# data processing

In [ ]:
df1 = pd.DataFrame(results)

preds2_mean_lst.insert(0, np.NaN)
preds2_mean_lst

preds_mean_lst = []
for i in range(len(df1)):
    if i == 0:
        preds_mean_lst.append(preds_mean)
    else:
        preds_mean_lst.append(np.nan)

len(preds_mean_lst)
preds_mean_lst


df1['preds_mean'] = preds_mean_lst
df1['preds2_mean'] = preds2_mean_lst

######################################################################
df_all_best = df1.copy()

base_value = df1.iloc[0,1]
df_all_best['baseline_bce'] = base_value
df_all_best
df_all_best['diff'] = df_all_best['bce']-df_all_best['baseline_bce']

base_preds = df1.iloc[0,3]
df_all_best['preds_mean'] = base_preds
df_all_best['diff_preds'] = df_all_best['preds2_mean'] - df_all_best['preds_mean']
df_all_best

df_all_best = df_all_best.drop(index=[0])
df_all_best = df_all_best.sort_values(by=['diff', 'diff_preds'])

#######################################################################

diffdf = df_all_best[['feature','diff']]
diffdf = diffdf.sort_values(by='diff')

diffpreds_df = df_all_best[['feature','diff_preds']]
diffpreds_df = diffpreds_df.sort_values(by='diff_preds')


# df_all_best.to_csv('df_all_best_for_meth1.csv',index = False)
# diffdf.to_csv('df_all_diff_for_meth1.csv',index = False)
# diffpreds_df.to_csv('df_all_preds_for_meth1.csv',index = False)

# ❗❗❗❗❗lib실행 후 여기부터!!!! data_load

In [2]:
all_best = pd.read_csv('/project/kelly/df_all_best_for_meth1.csv')
all_diff = pd.read_csv('/project/kelly/df_all_diff_for_meth1.csv')
all_preds = pd.read_csv('/project/kelly/df_all_preds_for_meth1.csv')
all_best_for_ReLU = pd.read_csv('/project/LSH/df_all_best.csv')

# ReLU

In [3]:
for i,j in enumerate(all_best_for_ReLU['diff']):
    if j <= 0:
        all_best_for_ReLU['diff'][i] = 0
all_best_for_ReLU

<ipython-input-3-86b9ebd9b2c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_best_for_ReLU['diff'][i] = 0


,feature,baseline_bce,bce,diff
0,51006,1.014362,1.001220,0.000000
1,63323026201,1.014362,1.006389,0.000000
2,51277,1.014362,1.006995,0.000000
3,0,1.014362,1.009931,0.000000
4,55390000401,1.014362,1.010688,0.000000
...,...,...,...,...
4063,10019055302,1.014362,1.041859,0.027498
4064,54817525,1.014362,1.043492,0.029130
4065,67434504,1.014362,1.046829,0.032467
4066,74606211,1.014362,1.051030,0.036668


# Sign

In [4]:
for i,j in enumerate(all_best['diff_preds']):
    if j>0:
        all_best['diff_preds'][i] = 1
#     elif j==0:
#         all_best['diff_preds'][i] = 0
    elif j<0:
        all_best['diff_preds'][i] = -1

<ipython-input-4-6c161a626d96>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_best['diff_preds'][i] = -1
<ipython-input-4-6c161a626d96>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_best['diff_preds'][i] = 1


# Method1
```
Method1 = all_best_for_ReLU['diff'] * all_best['diff_preds']

Method1 순서대로(내림차순) sorting
```

In [5]:
ReLU_sort = all_best_for_ReLU.sort_values('feature').reset_index(drop = True)
# ReLU_sort['diff']
Sign_sort = all_best.sort_values('feature').reset_index(drop = True)
# Sign_sort['diff_preds']

Method1 = ReLU_sort.copy()
Method1.drop(['baseline_bce', 'bce'], axis = 1, inplace = True)

Method1['diff_preds'] = Sign_sort['diff_preds']

Method1['Method1'] = ReLU_sort['diff'] * Sign_sort['diff_preds']
Method1 = Method1.sort_values('Method1', ascending = False)

In [6]:
# Method1.to_csv('Method1.csv', index = False)

# Graph
```
1. data (x_(7727,10,4068).npy, y_(7727,1))를 가져와서 사망, 생존자를 나눠. (.isin도 있음.)
2. feature importance 상위 10개 가져와.

```

In [7]:
x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

In [8]:
# 사망자 idx
dead_idx = np.where(y==1)
dead_idx

# 생존자 idx
surv_idx = np.where(y!=1)
surv_idx

(array([   0,    1,    2, ..., 7721, 7724, 7725]),)

(array([   3,   16,   18, ..., 7722, 7723, 7726]),)

In [9]:
surv_idx[0] # 0번째를 뽑아줘야 for문에 하나씩 들어가더라.

array([   3,   16,   18, ..., 7722, 7723, 7726])

In [10]:
# 사망자 list
dead_lst = []
for idx in dead_idx[0]:
    dead_lst.append(x[idx])

dead_lst


# 생존자 list
surv_lst = []
for idx in surv_idx[0]:
    surv_lst.append(x[idx])

surv_lst

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 arr

[array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.]]),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 arr

In [11]:
dead = np.array(dead_lst)
surv = np.array(surv_lst)
# dead
# surv

In [12]:
len(dead)
len(surv)

len(dead) + len(surv) == 7727


4718

3009

True

# FI는 Method1을 통해 구한 ReLU(diff) * Sign(diff_preds)
```
즉, Method1 df의 Method1열
```

In [13]:
method1 = pd.read_csv('Method1.csv')
method1

,feature,diff,diff_preds,Method1
0,409606211,0.059180,1.0,0.059180
1,74606211,0.036668,1.0,0.036668
2,67434504,0.032467,1.0,0.032467
3,54817525,0.029130,1.0,0.029130
4,10019055302,0.027498,1.0,0.027498
...,...,...,...,...
4063,51200,0.016966,-1.0,-0.016966
4064,182138167,0.017109,-1.0,-0.017109
4065,50889,0.018246,-1.0,-0.018246
4066,4003822,0.019987,-1.0,-0.019987


In [14]:
feat_lst = method1['feature'].to_list()

In [15]:
# 이렇게 하면 lab은 아예 안나옴. 그래서 lab, pre, pro 별로 top 10 구해야함.
# 그래서 total_data 갖다 쓰는 것.

# feat_top10 = method1['feature'][:10]
# feat_bottom10 = method1['feature'][-10:]

# feat_top10
# feat_bottom10

In [16]:
total_data = pd.read_csv('/project/LSH/total_data.csv')

total_data

,SUBJECT_ID,ITEMID,CHARTTIME,TYPE
0,9,50882,2149-11-09,LAB
1,9,50931,2149-11-09,LAB
2,9,50971,2149-11-09,LAB
3,9,51275,2149-11-09,LAB
4,9,51288,2149-11-09,LAB
...,...,...,...,...
9221915,99985,51288,2181-03-06,LAB
9221916,99985,51279,2182-03-14,LAB
9221917,99985,51006,2182-03-14,LAB
9221918,99985,51221,2182-03-14,LAB


In [17]:
LAB_ITEMID_arr = total_data[total_data['TYPE']=='LAB']['ITEMID'].unique()

LAB_feat_top10 = []
LAB_feat_bottom10 = []

for i in feat_lst:
    if i in LAB_ITEMID_arr:
        LAB_feat_top10.append(i)
        if len(LAB_feat_top10) == 10:
            break

len(LAB_feat_top10)


for i in feat_lst[::-1]:
    if i in LAB_ITEMID_arr:
        LAB_feat_bottom10.append(i)
        if len(LAB_feat_bottom10) == 10:
            break

len(LAB_feat_bottom10)

10

10

In [18]:
LAB_feat_top10

# 확인
total_data[total_data['ITEMID'].isin(LAB_feat_top10)]['TYPE'].unique()

[51003, 51148, 50868, 50813, 51120, 50908, 51482, 50917, 50862, 50911]

array(['LAB'], dtype=object)

## top10, bottom10 function

In [19]:
def topbottom(type_in_str):
    ITEMID_arr = total_data[total_data['TYPE'] == type_in_str]['ITEMID'].unique()
    feat_top10 = []
    feat_bottom10 = []

    for i in feat_lst:
        if i in ITEMID_arr:
            feat_top10.append(i)
            if len(feat_top10) == 10:
                break

    for i in feat_lst[::-1]:
        if i in ITEMID_arr:
            feat_bottom10.append(i)
            if len(feat_bottom10) == 10:
                break
    
    return ITEMID_arr, feat_top10, feat_bottom10

In [20]:
_, pre_top, pre_bottom = topbottom('PRE')
_, pro_top, pro_bottom = topbottom('PRO')
_, lab_top, lab_bottom = topbottom('LAB')

# len(pre_top)
# len(pre_bottom)
# len(pro_top)
# len(pro_bottom)
# len(lab_top)
# len(lab_bottom)

In [21]:
total_data[total_data['ITEMID'].isin(pre_bottom)]['TYPE'].unique()
# 근데 이거 유니크는 itemid 서로 안겹치니까 당연히 지들만 나오는건데 왜 확인한거지..? 흠흠...

array(['PRE'], dtype=object)

In [22]:
feat_lst[:10]

[409606211,
 74606211,
 67434504,
 54817525,
 10019055302,
 54858516,
 74706811,
 45006701,
 228125,
 63323016501]

In [23]:
pd.Series(feat_lst[:10]).isin(pro_top)
# .isin(~~)은 series나 df에만 쓰임, list나 ndarray에는 안쓰이므로 pd.Series나 pd.DataFrame를 통해 형변환해서 사용하자.
# .isin(~~)의 괄호 안에는 list, ndarray, Series 들어갈 수 있음.

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8     True
9    False
dtype: bool

In [24]:
def check_top(top):
    return pd.Series(feat_lst[:204]).isin(top)

def check_bottom(bottom):
    return pd.Series(feat_lst[::-1][:198]).isin(bottom)

In [25]:
check_top(lab_top).sum()
check_top(pre_top).sum()
check_top(pro_top).sum()
check_bottom(lab_bottom).sum()
check_bottom(pre_bottom).sum()
check_bottom(pro_bottom).sum()

10

10

10

10

10

10

In [26]:
lab_top

[51003, 51148, 50868, 50813, 51120, 50908, 51482, 50917, 50862, 50911]

In [27]:
lab_top_df = total_data[total_data['ITEMID'].isin(lab_top)]

In [28]:
len(lab_top_df)
lab_top_df[['SUBJECT_ID','CHARTTIME']]#.unique()

117456

,SUBJECT_ID,CHARTTIME
13,9,2149-11-09
52,9,2149-11-10
53,9,2149-11-10
101,9,2149-11-11
169,9,2149-11-13
...,...,...
9221132,99985,2181-01-29
9221188,99985,2181-01-30
9221363,99985,2181-02-02
9221420,99985,2181-02-03


In [29]:
lab_top_df.pivot_table

<bound method DataFrame.pivot_table of          SUBJECT_ID  ITEMID   CHARTTIME TYPE
13                9   50813  2149-11-09  LAB
52                9   50813  2149-11-10  LAB
53                9   50813  2149-11-10  LAB
101               9   50813  2149-11-11  LAB
169               9   50862  2149-11-13  LAB
...             ...     ...         ...  ...
9221132       99985   50862  2181-01-29  LAB
9221188       99985   50862  2181-01-30  LAB
9221363       99985   51482  2181-02-02  LAB
9221420       99985   51482  2181-02-03  LAB
9221825       99985   50862  2181-02-11  LAB

[117456 rows x 4 columns]>

In [30]:
# lab_top_df.groupby('SUBJECT_ID')['CHARTTIME']
lab_top_df.groupby('SUBJECT_ID')['CHARTTIME'].unique() #꺅,,, 이게되네..?
lab_top_df.groupby('SUBJECT_ID').count()

SUBJECT_ID
9        [2149-11-09, 2149-11-10, 2149-11-11, 2149-11-1...
21       [2134-09-11, 2134-09-12, 2134-09-13, 2134-09-1...
31                    [2108-08-22, 2108-08-26, 2108-08-29]
36       [2131-05-12, 2131-05-13, 2131-05-16, 2131-05-1...
37       [2183-08-21, 2183-08-22, 2183-08-23, 2183-08-2...
                               ...                        
99863                 [2142-04-13, 2142-04-25, 2142-05-03]
99934                             [2110-02-28, 2110-03-01]
99937                                         [2128-05-11]
99944                                         [2161-02-16]
99985    [2181-01-29, 2181-01-30, 2181-02-02, 2181-02-0...
Name: CHARTTIME, Length: 7314, dtype: object

,ITEMID,CHARTTIME,TYPE
SUBJECT_ID,,,
9,9,9,9
21,41,41,41
31,3,3,3
36,15,15,15
37,8,8,8
...,...,...,...
99863,3,3,3
99934,2,2,2
99937,1,1,1


근데 우리 30일, 20일 그거 안하기로 해서 이거 필요 없잖아..? 또 방향 잃고....

sub = pd.read_csv('/project/guri/data/폐렴환자.csv')
sub

sub['SUBJECT_ID'].nunique() #여기서 외래환자 제거한게 폐렴환자lab이었음.

생각해보면  뽑은게 이미 abnormal임...
그럼 걍 개수 세면 되는건가..?

In [31]:
lab_top_df.groupby(['ITEMID']).count()

,SUBJECT_ID,CHARTTIME,TYPE
ITEMID,,,
50813,24421,24421,24421
50862,34229,34229,34229
50868,18353,18353,18353
50908,2826,2826,2826
50911,6463,6463,6463
50917,1639,1639,1639
51003,23198,23198,23198
51120,742,742,742
51148,1428,1428,1428


## 버려..? 아니... 환자별로 D-10 ~ D-1로 날짜 처리 해줘야함.

In [32]:
# 함수 만들꺼면 잊지 말고 챙겨야할 cell
# 환자별 charttime.... 근데 우리 필요한건 itemid 별 charttime임... 내가 지금 환자별로 해서 뭐가 안되는 것 같음.
lab_top_times = lab_top_df.groupby(['SUBJECT_ID'])['CHARTTIME'].unique()

In [33]:
lab_top_times

SUBJECT_ID
9        [2149-11-09, 2149-11-10, 2149-11-11, 2149-11-1...
21       [2134-09-11, 2134-09-12, 2134-09-13, 2134-09-1...
31                    [2108-08-22, 2108-08-26, 2108-08-29]
36       [2131-05-12, 2131-05-13, 2131-05-16, 2131-05-1...
37       [2183-08-21, 2183-08-22, 2183-08-23, 2183-08-2...
                               ...                        
99863                 [2142-04-13, 2142-04-25, 2142-05-03]
99934                             [2110-02-28, 2110-03-01]
99937                                         [2128-05-11]
99944                                         [2161-02-16]
99985    [2181-01-29, 2181-01-30, 2181-02-02, 2181-02-0...
Name: CHARTTIME, Length: 7314, dtype: object

In [34]:
# 확인 cell
len(pd.DataFrame(lab_top_times).iloc[0][0])

5

In [35]:
# import datetime as dt
from datetime import datetime, timedelta

In [36]:
# 확인 cell
lab_top_times.index[0]

9

In [37]:
# cnt0 = 0
term = {}
for i,j in enumerate(lab_top_times):
    j = sorted(j)
    days = datetime.strptime(j[-1],'%Y-%m-%d').date() - datetime.strptime(j[0],'%Y-%m-%d').date()
#     days
    if days == timedelta(0):
#         cnt0 += 1
#         if i < 50:
#             j
        term[lab_top_times.index[i]] = {1:j}
#         pass
    else:
        days = int(str(days).split(' ')[0]) #사용자가 이해하기 쉬운형태로 문자열로
#         i,j, days
    #     repr(days) # 객체까지 문자열로
        term[lab_top_times.index[i]] = {days:j}
#     break

# cnt0
len(term) # 아... 0일이 빠진듯... ===> 0일 머무른거 추가해서 다시 했음!!!!!!!!

# cnt0 + len(term) # == 7314여야 함.
# https://wangin9.tistory.com/entry/datetime

7314

In [38]:
term[21]
# term[21].values() # 연속데이터가 아니네..?
val = term[21].values()
list(val)[0]

{148: ['2134-09-11',
  '2134-09-12',
  '2134-09-13',
  '2134-09-14',
  '2134-09-15',
  '2134-09-16',
  '2134-09-18',
  '2134-09-19',
  '2134-09-20',
  '2134-10-01',
  '2135-01-10',
  '2135-01-31',
  '2135-02-01',
  '2135-02-03',
  '2135-02-04',
  '2135-02-06']}

['2134-09-11',
 '2134-09-12',
 '2134-09-13',
 '2134-09-14',
 '2134-09-15',
 '2134-09-16',
 '2134-09-18',
 '2134-09-19',
 '2134-09-20',
 '2134-10-01',
 '2135-01-10',
 '2135-01-31',
 '2135-02-01',
 '2135-02-03',
 '2135-02-04',
 '2135-02-06']

In [39]:
lab_top_df[:10]

,SUBJECT_ID,ITEMID,CHARTTIME,TYPE
13,9,50813,2149-11-09,LAB
52,9,50813,2149-11-10,LAB
53,9,50813,2149-11-10,LAB
101,9,50813,2149-11-11,LAB
169,9,50862,2149-11-13,LAB
170,9,50862,2149-11-13,LAB
200,9,50911,2149-11-13,LAB
201,9,50911,2149-11-13,LAB
238,9,50911,2149-11-14,LAB
320,21,50908,2134-09-11,LAB


In [40]:
lab_item = lab_top_df.groupby(['SUBJECT_ID', 'CHARTTIME'])['ITEMID'].unique()

In [41]:
pd.DataFrame(lab_item)

ITEMID
SUBJECT_ID CHARTTIME                 
9          2149-11-09         [50813]
           2149-11-10         [50813]
           2149-11-11         [50813]
           2149-11-13  [50862, 50911]
           2149-11-14         [50911]
...                               ...
99985      2181-01-29  [50813, 50862]
           2181-01-30         [50862]
           2181-02-02         [51482]
           2181-02-03         [51482]
           2181-02-11         [50862]

[66316 rows x 1 columns]

In [42]:
# 확인 cell
lab_top_df['ITEMID'].nunique() #이미 10개인데.... 아니 d-10~ d-1까지만 하면 될 것 같은데...

10

In [43]:
lab_top_times

SUBJECT_ID
9        [2149-11-09, 2149-11-10, 2149-11-11, 2149-11-1...
21       [2134-09-11, 2134-09-12, 2134-09-13, 2134-09-1...
31                    [2108-08-22, 2108-08-26, 2108-08-29]
36       [2131-05-12, 2131-05-13, 2131-05-16, 2131-05-1...
37       [2183-08-21, 2183-08-22, 2183-08-23, 2183-08-2...
                               ...                        
99863                 [2142-04-13, 2142-04-25, 2142-05-03]
99934                             [2110-02-28, 2110-03-01]
99937                                         [2128-05-11]
99944                                         [2161-02-16]
99985    [2181-01-29, 2181-01-30, 2181-02-02, 2181-02-0...
Name: CHARTTIME, Length: 7314, dtype: object

## 다시, 아.... zero array....

### lab_top_times 바뀜...

In [44]:
lab_top_times = lab_top_df.groupby(['ITEMID'])['CHARTTIME'].unique() #아 내가 이걸 설마 바꿨었구나...

#### 바로 아래 cell 버려

In [45]:
# 버..려... test cell...
# term = {}
for i,j in enumerate(lab_top_times):
    j = sorted(j)
    j
    dday = datetime.strptime(j[-1],'%Y-%m-%d').date()
#     dday
    
    days = dday - datetime.strptime(j[0],'%Y-%m-%d').date()
    
    d10 = (dday - timedelta(10)).strftime('%Y-%m-%d')
#     d10
    
    if d10 in j:
        'yes'
    else:
        for d in list(range(1,11))[::-1]:
            dday - timedelta(d)
            if (dday - timedelta(d)).strftime('%Y-%m-%d') in j:
                'yes'
            else:
                'no'
#     if days == timedelta(0):
#         pass
# #         term[lab_top_times.index[i]] = {1:j}
# #         lab_top_times.index[i]
#     else:
#         days = int(str(days).split(' ')[0]) #사용자가 이해하기 쉬운형태로 문자열로
#         days
# #         term[lab_top_times.index[i]] = {days:j}
    break

['2100-06-15',
 '2100-07-02',
 '2100-09-06',
 '2100-09-08',
 '2100-09-14',
 '2100-09-15',
 '2100-09-16',
 '2100-09-18',
 '2100-09-19',
 '2100-09-20',
 '2100-09-21',
 '2100-09-24',
 '2100-10-06',
 '2100-10-07',
 '2100-10-14',
 '2100-10-17',
 '2100-10-21',
 '2100-10-22',
 '2100-10-23',
 '2100-10-24',
 '2100-10-29',
 '2100-11-02',
 '2100-11-13',
 '2100-11-14',
 '2100-11-16',
 '2100-11-20',
 '2100-11-21',
 '2100-11-22',
 '2100-11-23',
 '2100-11-24',
 '2100-11-25',
 '2100-11-27',
 '2100-11-29',
 '2100-12-01',
 '2100-12-03',
 '2100-12-15',
 '2101-01-01',
 '2101-01-13',
 '2101-01-16',
 '2101-01-18',
 '2101-02-19',
 '2101-03-10',
 '2101-03-11',
 '2101-03-20',
 '2101-03-27',
 '2101-03-29',
 '2101-04-01',
 '2101-04-08',
 '2101-04-09',
 '2101-04-12',
 '2101-04-14',
 '2101-04-15',
 '2101-04-17',
 '2101-04-20',
 '2101-04-27',
 '2101-04-29',
 '2101-05-18',
 '2101-05-25',
 '2101-05-31',
 '2101-06-02',
 '2101-06-06',
 '2101-06-07',
 '2101-06-14',
 '2101-06-15',
 '2101-07-05',
 '2101-07-09',
 '2101-07-

datetime.date(2209, 7, 22)

'no'

datetime.date(2209, 7, 23)

'no'

...

'no'

datetime.date(2209, 7, 30)

'no'

datetime.date(2209, 7, 31)

'yes'

## 다시, 아... zero array.... 이어서...

In [46]:
# 일단 환자별로 d-1~d-10 존재하면 해당 날짜 row에 1로 표시.

lab_top_arr = np.zeros((lab_top_df['SUBJECT_ID'].nunique(), 10, 10))

for i,j in enumerate(lab_top_times):
    j = sorted(j)
#     j
    dday = datetime.strptime(j[-1],'%Y-%m-%d').date()
    
    days = dday - datetime.strptime(j[0],'%Y-%m-%d').date()
    
    cnt = 0
    for d in list(range(1,11))[::-1]: #d-10부터 적혀야함. cnt로 해결!
#         dday - timedelta(d)
        if (dday - timedelta(d)).strftime('%Y-%m-%d') in j:
            lab_top_arr[i,cnt] = 1 #환자의 day시점을 1로 채워주기.
            cnt += 1
        else:
            cnt += 1
            pass
#     break
# cnt
# lab_top_arr[0] #0번째 환자 확인
lab_top_arr

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [47]:
lab_item_df = pd.DataFrame(lab_item)#.index()#('CHARTTIME')
lab_item_df.index #아.... .index()는 안되고 .index는 되고..?ㅋ...

MultiIndex([(    9, '2149-11-09'),
            (    9, '2149-11-10'),
            (    9, '2149-11-11'),
            (    9, '2149-11-13'),
            (    9, '2149-11-14'),
            (   21, '2134-09-11'),
            (   21, '2134-09-12'),
            (   21, '2134-09-13'),
            (   21, '2134-09-14'),
            (   21, '2134-09-15'),
            ...
            (99863, '2142-05-03'),
            (99934, '2110-02-28'),
            (99934, '2110-03-01'),
            (99937, '2128-05-11'),
            (99944, '2161-02-16'),
            (99985, '2181-01-29'),
            (99985, '2181-01-30'),
            (99985, '2181-02-02'),
            (99985, '2181-02-03'),
            (99985, '2181-02-11')],
           names=['SUBJECT_ID', 'CHARTTIME'], length=66316)

In [48]:

lab_item_df.iloc[3]
lab_item_df

ITEMID    [50862, 50911]
Name: (9, 2149-11-13), dtype: object

ITEMID
SUBJECT_ID CHARTTIME                 
9          2149-11-09         [50813]
           2149-11-10         [50813]
           2149-11-11         [50813]
           2149-11-13  [50862, 50911]
           2149-11-14         [50911]
...                               ...
99985      2181-01-29  [50813, 50862]
           2181-01-30         [50862]
           2181-02-02         [51482]
           2181-02-03         [51482]
           2181-02-11         [50862]

[66316 rows x 1 columns]

In [50]:
cnt = 0 # 전체 큰 index용...
for i,j in enumerate(lab_top_times):
    cnt_day = 0
    for k in range(len(j)):
        lab_item_df.index[cnt][1]
        lab_item_df.index[cnt]
#         lab_item_df.iloc[cnt].values[0][0] #itemid
        idx = lab_top.index(lab_item_df.iloc[cnt].values[0][0]) #itemid의 lab feat내 FI 순서
        idx
#         lab_top_arr[i][cnt_day][idx]
        cnt += 1
        break
    break
# lab_item_df.index[0][1]
cnt

'2149-11-09'

(9, '2149-11-09')

3

1

## 어렵게 생각하지 말고 다시........

In [51]:
lab_item_df.iloc[0]

ITEMID    [50813]
Name: (9, 2149-11-09), dtype: object

In [55]:
lab_top_times

ITEMID
50813    [2149-11-09, 2149-11-10, 2149-11-11, 2134-09-1...
50862    [2149-11-13, 2134-09-19, 2134-09-20, 2135-01-1...
50868    [2134-09-11, 2134-09-12, 2134-09-13, 2134-09-1...
50908    [2134-09-11, 2134-09-12, 2134-09-13, 2134-09-1...
50911    [2149-11-13, 2149-11-14, 2134-09-11, 2134-09-1...
50917    [2174-01-02, 2174-01-04, 2174-01-08, 2117-01-0...
51003    [2134-09-11, 2131-05-12, 2131-05-13, 2131-05-1...
51120    [2141-09-08, 2141-10-13, 2141-10-19, 2141-11-1...
51148    [2167-07-25, 2167-07-26, 2136-03-22, 2130-08-0...
51482    [2173-08-10, 2196-09-26, 2196-09-28, 2189-01-2...
Name: CHARTTIME, dtype: object

In [54]:
lab_top_arr = np.zeros((lab_top_df['SUBJECT_ID'].nunique(), 10, 10))

for i,j in enumerate(lab_top_times):
    j = sorted(j)
    'j',j
    dday = datetime.strptime(j[-1],'%Y-%m-%d').date()
    
    cnt = 0
    for d in list(range(1,11))[::-1]: #d-10부터 적혀야함. cnt로 해결!
        tt = (dday - timedelta(d)).strftime('%Y-%m-%d')
        if tt in j:
            lab_top_arr[i,cnt] = 1 #환자의 day시점을 1로 채워주기.
####################################################################################################
            lab_item_df.iloc[i].index(tt)
####################################################################################################            
            cnt += 1
        else:
            cnt += 1
            pass
    break
# cnt
lab_top_arr[0] #0번째 환자 확인
# lab_top_arr

('j',
 ['2100-06-15',
  '2100-07-02',
  '2100-09-06',
  '2100-09-08',
  '2100-09-14',
  '2100-09-15',
  '2100-09-16',
  '2100-09-18',
  '2100-09-19',
  '2100-09-20',
  '2100-09-21',
  '2100-09-24',
  '2100-10-06',
  '2100-10-07',
  '2100-10-14',
  '2100-10-17',
  '2100-10-21',
  '2100-10-22',
  '2100-10-23',
  '2100-10-24',
  '2100-10-29',
  '2100-11-02',
  '2100-11-13',
  '2100-11-14',
  '2100-11-16',
  '2100-11-20',
  '2100-11-21',
  '2100-11-22',
  '2100-11-23',
  '2100-11-24',
  '2100-11-25',
  '2100-11-27',
  '2100-11-29',
  '2100-12-01',
  '2100-12-03',
  '2100-12-15',
  '2101-01-01',
  '2101-01-13',
  '2101-01-16',
  '2101-01-18',
  '2101-02-19',
  '2101-03-10',
  '2101-03-11',
  '2101-03-20',
  '2101-03-27',
  '2101-03-29',
  '2101-04-01',
  '2101-04-08',
  '2101-04-09',
  '2101-04-12',
  '2101-04-14',
  '2101-04-15',
  '2101-04-17',
  '2101-04-20',
  '2101-04-27',
  '2101-04-29',
  '2101-05-18',
  '2101-05-25',
  '2101-05-31',
  '2101-06-02',
  '2101-06-06',
  '2101-06-07',
  

TypeError: 'Index' object is not callable

In [470]:
lab_top_arr[0][0]#[]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [466]:
lab_top # 이건 중요한 순서니까 순서 바뀌면 안됨. 정렬 금지.
# sorted(lab_top)
# np.array(lab_top)[np.where(51148)]
lab_top.index(51003) # 위치 반환

[51003, 51148, 50868, 50813, 51120, 50908, 51482, 50917, 50862, 50911]

0

In [369]:
list(range(1,11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### shape = (환자nunique, 10, 10)으로 np.zeros 만들기...
### dday를 기준으로 d-10이 있으면 1로 채우기, 없으면 0 (냅두기)

In [398]:
lab_top_arr = np.zeros((lab_top_df['SUBJECT_ID'].nunique(), 10, 10))

In [399]:
lab_top_arr.shape

(7314, 10, 10)

In [397]:
# # test cell
# lab_top_arr[0]# 맨 앞 환자
# lab_top_arr[0,0] # day 한 줄
# lab_top_arr[0,0] = 1 # day 한 줄 1로 채움.
# lab_top_arr[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [235]:
# test cell
datetime.today().date()-timedelta(1)

datetime.date(2022, 2, 12)

TypeError: unsupported type for timedelta days component: datetime.date

In [142]:
ls

'01-1. EDA 및 환자 추출&Labevents 원본.ipynb'
'01-2. EDA 및 환자 추출 코드 with open으로 다시.ipynb'
'01-3. EDA 및 환자 추출 코드 with open + 함수로 작성 후 정리.ipynb'
'02-2. LSTM,RF Seed Test.ipynb'
'02. 다른 Features 추가, 새로운 DF 생성 (데이터 가공 & 모델링).ipynb'
'02. 다른 Features 추가, 새로운 DF 생성 (데이터 가공 & 모델링) with open으로 가공중,,, 반포기...-Copy1.ipynb'
'02. 다른 Features 추가, 새로운 DF 생성 (데이터 가공 & 모델링) with open으로 가공중,,, 반포기....ipynb'
'03. 최종 LSTM + VotingClassifier guri.ipynb'
'03 시계열, 비시계열 modeling 및 stability 비교 LSH.ipynb'
'03. 기존 VotingClassifier + Feature Selection.ipynb'
'04 Keras Tuner LSH.ipynb'
'05-1. FI guriguri.ipynb'
'05-1. FI guri.ipynb'
'05. LSTM Feature Importance guri.ipynb'
'05 Voting Classifier LSH.ipynb'
'06-1. RESET.ipynb'
'06-2. RESET with open으로 다시.ipynb'
'06. LSTM + Adaboost guri1.ipynb'
'06. LSTM + Adaboost guri2.ipynb'
'11. 3Tables-final 포함.ipynb'
'11. 3Tables-final 포함 with.ipynb'
'12. Multi Output (ft.window)+final.ipynb'
'12. Multi Output (ft.window).ipynb'
'13. Expanded Data (ft. window).ipynb'
'14. 최신 딥러닝 모델 적

In [144]:
pwd

'/project/kelly'

NameError: name 'export' is not defined